# Исследование надёжности заёмщиков

# Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента, а также, уровень дохода и цели кредита на факт погашения задолженности в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Цель исследования

*Получить ответы на поставленные вопросы:*
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?


# Описание данных:

*Датасет data.csv*

**children** — количество детей в семье  
**days_employed** — общий трудовой стаж в днях  
**dob_years** — возраст клиента в годах  
**education** — уровень образования клиента  
**education_id** — идентификатор уровня образования  
**family_status** — семейное положение  
**family_status_id** — идентификатор семейного положения  
**gender** — пол клиента  
**income_type** — тип занятости  
**debt** — имел ли задолженность по возврату кредитов  
**total_income** — ежемесячный доход  
**purpose** — цель получения кредита

# Содержание:

1 Обзор данных  
2 Предобработка данных  
2.1 Обработка пропусков  
2.2 Замена типа данных  
2.3 Обработка дубликатов  
2.4 Лемматизация  
2.5 Категоризация данных  
3 Ответы на поставленные вопросы  
3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?  
3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?  
3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?  
3.4 Как разные цели кредита влияют на его возврат в срок?  
4 Общий вывод  
  
  
  

## 1 Обзор данных

Для начала загрузим библиотеки, которые понадобятся для работы над проектом.

In [1]:
import pandas as pd

from pymystem3 import Mystem
m = Mystem()

from collections import Counter

Теперь прочитаем файл с данными и сохраним его для дальнейшей работы в отдельной переменной `df`

In [2]:
df = pd.read_csv('/datasets/data.csv')

Для ознакомления с данными, выведем последние 15 строк датафрейма, общую информацию о нем, а также воспользуемся методом `describe()` для знакомства с числовыми значениями. 

In [3]:
display(df.tail(15))
df.info()
df.describe()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**На этапе обзора данных обнаружены некорректные значения:**
1. В столбце `children` есть отрицательное значение (похоже на ошибку), максимальное значение 20 также похоже на ошибку.
2. Данные по трудовому стажу в столбце `days_employed` содержат пропуски, отрицательные значения, а также аномально высокие положительные значения, что говорит о возможном использовании двух разных источников для наполнения датасета.  Также, в столбце используются вещественные числа, которые для удобства можно заменить на целочисленные значения.
3. В столбце `dob_years` минимальное значение 0, хотя возраст заемщика должен быть не менее 18 лет.
3. При заполнении  столбца `education` использованы разные регистры букв, есть дубликаты.
4. В столбце `total_income` есть пропуски, данные представлены в виде вещественных чисел, которые для удобства восприятия можно перевести в целочисленные значения.
5. В столбце `purpose` слишком много вариаций целей получения кредита, многие из которых имеют сходные смысловые значения.

**Вывод:**
Для дальнейшей работы над проектом необходимо устранить проблемы в данных.

## 2 Предобработка данных

### 2.1 Обработка пропусков

Для начала проверим количество строк с некорректными значениями в столбце cnildren:

In [4]:
print('Количество строк с отрицательным значением (-1):', df[df['children'] == -1].count()[0])
print('Количество строк с аномально высоким значением (20):', df[df['children'] == 20].count()[0])

Количество строк с отрицательным значением (-1): 47
Количество строк с аномально высоким значением (20): 76


Относительно общего количества строк эти данные составляют менее 0.5%, поэтому их можно исправить на основе предположения:
Данные могли быть сформированы из разных источников, где -1 могло означать статус отсутствия детей, поэтому исправляем на 0, а 20 больше соответствует 2, поэтому исправляем на 2. На статистику изменения сильно не повлияют, но данные теперь будут более правдоподобные.

In [5]:
df['children'] = df['children'].replace(-1, 0)  # производим замену данных
df['children'] = df['children'].replace(20, 2)

In [6]:
print(df['children'].unique())  # контролируем результат

[1 0 3 2 4 5]


Следующим шагом, проверим количество строк c нулевым значением в столбце `dob_years`, проведем замену средним арифмитическим значением (с округлением до целочисленного значения) и проверим результат, а также наличие других некорректных значений (меньше 18 лет):

In [7]:
print('Количество строк с нулевым значением:', df[df['dob_years'] == 0].count()[0])  # выводим строки с нулевым значением

Количество строк с нулевым значением: 101


In [8]:
df.loc[df['dob_years'] == 0, 'dob_years'] = int(df['dob_years'].mean())  # проводим замену средним арифметическим значением

In [9]:
print('Количество строк с возрастом < 18 лет:', df[df['dob_years'] < 18].count()[0])  # контролируем результат

Количество строк с возрастом < 18 лет: 0


Теперь переходим к корректировке данных столбца `days_employed`. Судя по отрицательным и положительным значением, а также наличием аномально высоких значений, столбец `days_employed` вероятно был сформирован из двух различных баз данных, в одной из которых указывалось значение в днях и с отрицательным значением (возможно знак тире принял значение минуса), а в другой (с аномально высокими значениями) - фиксировались значения в часах.  С учетом данных выводов, принимаем максимальное значение в стажа в днях  в количестве 30_000 и исправляем значения, используя следующий код:

In [10]:
df['days_employed'] = df['days_employed'].apply(abs)  # приводим отрицательные значения к положительным
df.loc[df['days_employed'] >30000, 'days_employed'] = df.loc[df['days_employed'] >30000, 'days_employed'] / 24  # переводим часы в дни

Заполняем пропуски в данных по стажу в столбце `days_employed` медианными значениями по возрастным группам:

In [11]:
print('Пропуски до:', df['days_employed'].isna().sum())  # выводим количество пропусков

Пропуски до: 2174


In [12]:
df['days_employed'] = df['days_employed'].fillna(df.groupby('dob_years')['days_employed'].transform('median'))  # заполняем пропуски

In [13]:
print('Пропуски после:', df['days_employed'].isna().sum())  # контролируем результат

Пропуски после: 0


Теперь переходим к столбцу `total_income`, заполняем и проверяем пропуски в данных медиаными значениями в соответствии с типом занятости:

In [14]:
print('Пропуски до:', df['total_income'].isna().sum())  # выводим количество пропусков

Пропуски до: 2174


In [15]:
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))  # заменяем пропуски

In [16]:
print('Пропуски после:', df['total_income'].isna().sum())  # контролируем результат

Пропуски после: 0


**Вывод** 

Пропуски и ошибки данных в столбцах `children`, `days_employed`, `dob_years` и `total_income` исправлены, теперь они однородны и готовы к дальнейшей работе над проектом.

### 2.2 Замена типа данных

Заменим вещественные числа (float) в столбцах `days_employed` и `total_income` на целочисленные значения (int). Поскольку работа с заменой пропусков уже проведена, можно использовать метод `astype()`, он сработает в данном случае без ошибок и без применения конструкции try - except.

In [17]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

In [18]:
df.head()  # выводим результат

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Все числовые данные теперь имют тип int и стали удобнее для восприятия.

### 2.3 Обработка дубликатов

Для начала приведем все значения в столбце education к одному регистру и проверим результат:

In [19]:
print('До обработки:', '\n', df['education'].value_counts())  # выводим список значений до обработки

До обработки: 
 среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [20]:
df['education'] = df['education'].str.lower()  # приводим значения к одному регистру

In [21]:
print('После обработки:', '\n', df['education'].value_counts())  # контролируем результат

После обработки: 
 среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Теперь избавимся от дубликатов и проверим результат:

In [22]:
print('Кол-во дубликатов до обработки:', df.duplicated().sum())  # проверяем кол-во дубликатов до обработки

Кол-во дубликатов до обработки: 71


In [23]:
df = df.drop_duplicates().reset_index(drop=True)  # удаляем дубликаты

In [24]:
print('Кол-во дубликатов после обработки:', df.duplicated().sum())  # контролируем результат

Кол-во дубликатов после обработки: 0


**Вывод**

Дубликаты могли появиться в результате ошибок и сбоев при формировании датафрейма. Теперь они устранены.

### 2.4 Лемматизация

Для начала выведем список уникальных значений столбца `purpose`

In [25]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Вариантов целей со сходными смысловыми значениями действительно много, поэтому необходимо лемматизировать данные столбца purpose, выбрать основные цели получения кредита, создать новый столбец `purpose_categorized` с обобщенными категориями целей и заполнить его данными с помощью функции.

In [26]:
lemmas_list = []
for element in df['purpose']:
    lemma = m.lemmatize(element)
    lemmas_list.extend(lemma)

Теперь создадим список лемм с подсчетом уникальных значений и отсортируем по количеству в порядке убывания:

In [27]:
unique_lemmas = Counter(lemmas_list)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

На основе представленных данных, видно, что здесь можно выделить четыре основные категории: недвижимость (жилье относим сюда же), автомобиль, образование и свадьба. Остается написать функцию для категоризации. Пропишем условия if для лемм 'автомобиль', 'образование' и 'свадьба', остальное отнесем к самой многочисленной категории - 'недвижимость'.

In [28]:
def new_category(row):
    lemmas = m.lemmatize(row)
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    return 'недвижимость'
df['purpose_categorized'] = df['purpose'].apply(new_category)

Для проверки результат выведем на экран первые 5 строк таблицы

In [29]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categorized
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

Теперь цели получения кредита уместились в четыре основные категории и данные стали более удоными для проведения дальнейшего анализа.

### 2.5 Категоризация данных

Создадим новые датафреймы-словари education_dict и family_status_dict и проведем визуальную проверку корректности пар в столбцах education - education_id и family_status - family_status_id:

In [30]:
education_dict = df[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

In [31]:
education_dict  # проводим визуальную проверку

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [32]:
family_status_dict = df[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

In [33]:
family_status_dict  # проводим визуальную проверку

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Словари созданы, наименования и числовые значения категорий корректны.

Для категоризации клиентов банка по уровню дохода, определим необходимые диапазоны значений. Для этого можно воспользоваться процентилями, использовав метод `describe()`, или дискретизацию на основе квантилей `qcut()`.

In [34]:
income_quant = pd.qcut(df['total_income'], 4)
income_quant.value_counts()

(107623.0, 142594.0]      5479
(20666.999, 107623.0]     5364
(195820.25, 2265604.0]    5364
(142594.0, 195820.25]     5247
Name: total_income, dtype: int64

Теперь с помощью функции проведем категоризацию и запишем данные в новый столбец income_categorized:

In [35]:
def income_categorized(row):
    if row <= 107623:
        return 'низкий'
    elif 107623 < row <= 142594:
        return 'средний'
    elif 195820 < row <= 2265604:
        return 'выше среднего'
    else:
        return 'высокий'
df['income_categorized'] = df['total_income'].apply(income_categorized)

In [36]:
df.head(10)  # проверка таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categorized,income_categorized
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,выше среднего
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,высокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,выше среднего
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,выше среднего
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,выше среднего
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,высокий


**Вывод**

Категоризация клиентов по уровню дохода завершена, ее можно было провести, используя другие критерии, к примеру на основе каких-либо статистических данных. Но в данном случае, категории выделены на основе распределения данных, а не фактических числовых границ интервалов.

## 3 Ответы на поставленные вопросы

### 3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на вопрос, создаем таблицу, сгруппированную по количеству детей, где 'count' - количество клиентов данной группы, 'sum' - количество клиентов, нарушивших срок оплаты по кредиту, a 'percent' - их соотношение в процентах.

In [37]:
children_debt = pd.pivot_table(df,
                              index=['children'],
                              values=['debt'],
                              aggfunc=['count', 'sum'])
children_debt['percent'] = round(children_debt['sum'] / children_debt['count'] * 100, 2)
display(children_debt.sort_values('percent'))

,count,sum,percent
,debt,debt,
children,,,
5,9,0,0.00
0,14138,1064,7.53
3,330,27,8.18
1,4808,444,9.23
2,2128,202,9.49
4,41,4,9.76


**Вывод**

Зависимость между наличием детей и возвратом кредита в срок прослеживается. В соответствии с приведенными данными, наличие детей увеличивает вероятность появления просроченной задолженности. Вероятно, это связано с возрастанием финансовой нагрузки при увеличении количества детей в семье. Интересная особенность прослеживается у клиентов с 5-ю детьми - они не имеют просрочек. При этом, по данной категории данные нельзя считать репрезентативными, поскольку таких клиентов всего 9. Клиенты с тремя детьми также выделяются из общего тренда - они имеют меньшую вероятность задержки платежей в сравнении с клиентами, у которых один, два или четыре ребенка. Возможно, это связано с наличием некоторых льгот для многодетных семей, которые начинают получать семьи с 3-мя детьми.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех: Вывод не противоречит полученному результату. 

</div>

### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на вопрос, создаем таблицу, сгруппированную по категории семейного положения, где 'count' - количество клиентов данной группы, 'sum' - количество клиентов, нарушивших срок оплаты по кредиту, a 'percent' - их соотношение в процентах.

In [38]:
family_status_debt = pd.pivot_table(df,
                              index=['family_status'],
                              values=['debt'],
                              aggfunc=['count', 'sum'])
family_status_debt['percent'] = round(family_status_debt['sum'] / family_status_debt['count'] * 100, 2)
display(family_status_debt.sort_values('percent'))

,count,sum,percent
,debt,debt,
family_status,,,
вдовец / вдова,959,63,6.57
в разводе,1195,85,7.11
женат / замужем,12339,931,7.55
гражданский брак,4151,388,9.35
Не женат / не замужем,2810,274,9.75


**Вывод**

Люди, никогда не имевшие законных отношений, более склонны к нарушению обязательств по срокам оплаты кредита. Согласно полученным данным, просрочки платежей допускают более 9.35% людей, относящихся к категориям "гражданский брак" и  "Не женат, не замужем", в то время, как среди клиентов других категорий нарушения допускают не более 7,55% заемщиков. Самый низкий показатель должников (6,57%) наблюдается среди клиентов, относящихся к категории "вдова/вдовец". Таким образом, наличие документа о законном браке (в настоящем или прошлом) может быть положительно оценено при выдаче кредита.

### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на вопрос, создаем таблицу, сгруппированную по категориям уровня дохода, где 'count' - количество клиентов данной группы, 'sum' - количество клиентов, нарушивших срок оплаты по кредиту, a 'percent' - их соотношение в процентах.

In [39]:
income_categorized_debt = pd.pivot_table(df,
                              index=['income_categorized'],
                              values=['debt'],
                              aggfunc=['count', 'sum'])
income_categorized_debt['percent'] = round(income_categorized_debt['sum'] / income_categorized_debt['count'] * 100, 2)
display(income_categorized_debt.sort_values('percent'))

,count,sum,percent
,debt,debt,
income_categorized,,,
выше среднего,5364,383,7.14
низкий,5364,427,7.96
высокий,5247,448,8.54
средний,5479,483,8.82


**Вывод**

Самый низкий показатель должников наблюдается в группе с доходом выше среднего. В группах с высоким и средним доходом заемщики выглядят наименее дисциплинированными. Возможно, со стороны клиентов имеет место завышение уровня доходов при подаче заявки на кредит.

### 3.4 Как разные цели кредита влияют на его возврат в срок?

Для ответа на вопрос, создаем таблицу, сгруппированную по целям получения кредита, где 'count' - количество клиентов данной группы, 'sum' - количество клиентов, нарушивших срок оплаты по кредиту, a 'percent' - их соотношение в процентах.

In [40]:
purpose_debt = pd.pivot_table(df,
                              index=['purpose_categorized'],
                              values=['debt'],
                              aggfunc=['count', 'sum'])
purpose_debt['percent'] = round(purpose_debt['sum'] / purpose_debt['count'] * 100, 2)
display(purpose_debt.sort_values('percent'))

,count,sum,percent
,debt,debt,
purpose_categorized,,,
недвижимость,10811,782,7.23
свадьба,2324,186,8.00
образование,4013,370,9.22
автомобиль,4306,403,9.36


**Вывод**

Клиенты, получающие кредиты, связанные с недвижимостью, представлены в самой многочисленной категории, и при этом являются самыми ответственными заемщиками. Самый высокий процент просрочек наблюдается среди клиентов, указавших целью кредита приобретение автомобиля.

## 4 Общий вывод

В рамках данного проекта обнаружены и устранены проблемы в данных, проведена категоризация клиентов, на основании которой удалось установить зависимости по возврату кредита в срок во всех категориях. Установлено, что самыми ответственными заемщиками являются клиенты без детей, при этом когда-либо состоящие в законном браке, с доходом выше среднего и получающие кредиты, связанные с недвижимостью. И, напротив, банку стоит проявлять осторожность при выдаче кредитов клиентам с детьми, людям, никогда не состоявшим в законных отношениях, тщательнее проверять документы, подтверждающие высокий уровень дохода и оценивать дополнительные риски, когда целью получения кредита является покупка автомобиля.